## Operator overloading

In [1]:
# Unary Operator
## - __neg__ if x is -2 then -x == 2
## + __pos__ x == +x
## ~ __invert__  if x is 2 then ~x == -3

#vetcor_v6.py

def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

def __neg__(self):
    return Vector(-x for x in self)

def __pos__(self):
    return Vector(self)

In [38]:
#! -*- coding: utf-8 -*-
#!/usr/bin/env python3
# vector_v6.py
# operator overloading
from array import array
import reprlib  #reprlib.repr
import math
import numbers  #numbers.Integral
import functools  #reduce,map
import operator  #xor
import itertools

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        # reprlib.repr used to present some elaboration that produces safe
        # format of large recursive iterations by limiting the length of
        # the output string and marking the cut with '...'
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes(self):
        return (bytes([ord(self.typecode)]) +
               bytes(self._components))

    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))

    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    # Supporting the sequence protocol
    def __len__(self):
        return len(self._components)

    # to support iteration, __getitem__ is required
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes ‘a’ to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(x, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)
    
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
    
    def __rmul__(self, scalar):
        return self * scalar
    
    def __matmul__(self, other):
        try:
            return sum(a * b for a, b in zip(self, other))
        except TypeError:
            return NotImplemented
    
    def __rmatmul__(self, other):
        return self @ other
    
    def __eq__(self, other):
        return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))

    

In [33]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])

In [21]:
v1 + v2

Vector([9.0, 11.0, 13.0])

In [22]:
v2 + [4, 5, 6]

Vector([10.0, 12.0, 14.0])

In [35]:
11 * v1

Vector([33.0, 44.0, 55.0])

In [37]:
v1 * 12

Vector([36.0, 48.0, 60.0])

In [45]:
v1 @ 3

TypeError: unsupported operand type(s) for @: 'Vector' and 'int'

In [42]:
v1 + (3,3,3) == v2

True